In [ ]:
import numpy as np
import keras.backend as K

In [ ]:
IM_HEIGHT = 225
IM_WIDTH = 300
BATCH_SIZE = 16
SEED = 473536
akiec_location = '/home/ubuntu/Data/Skin/akiec_others/'

np.random.seed(SEED)

In [ ]:
from PIL import Image

mean_im = Image.fromarray(np.load(akiec_location + 'mean_image_akiec_others.npz')['image']
                          .astype(np.uint8)).resize((IM_WIDTH, IM_HEIGHT))

mean_img = np.asarray(mean_im).astype(np.float)

In [ ]:
def preprocess_image(im):
    return (im - mean_img)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import compute_class_weight

img_gen = ImageDataGenerator(preprocessing_function=preprocess_image, rescale=1./255)

train_gen = img_gen.flow_from_directory(akiec_location + 'train',
                                       batch_size=BATCH_SIZE, shuffle=True,
                                       seed=SEED, target_size=(IM_HEIGHT, IM_WIDTH),
                                        classes=['NON_AKIEC', 'AKIEC'],
                                       class_mode='binary')

val_gen = img_gen.flow_from_directory(akiec_location + 'val',
                                       batch_size=BATCH_SIZE, shuffle=True,
                                       seed=SEED, target_size=(IM_HEIGHT, IM_WIDTH),
                                      classes=['NON_AKIEC', 'AKIEC'],
                                       class_mode='binary')

test_gen = img_gen.flow_from_directory(akiec_location + 'test',
                                       batch_size=BATCH_SIZE, shuffle=False,
                                       seed=SEED, target_size=(IM_HEIGHT, IM_WIDTH),
                                       classes=['NON_AKIEC', 'AKIEC'],
                                       class_mode='binary')

y_train = train_gen.classes
class_weight = compute_class_weight('balanced', np.unique(y_train), y_train.ravel())
class_weights = {i: class_weight[i] for i in range(len(class_weight))}

print('Class weights: ', class_weights)
class_weights[1] += 2.
print('Class weights: ', class_weights)

In [ ]:

data_name = 'akiec_non_akiec'
model_name = 'inceptionv3'
num_hidden_units = 512

num_hidden_units_1 = 512
num_hidden_units_2 = 512

filename = 'final' + data_name + '_' + model_name + '_' + str(num_hidden_units_1) + '_' + str(num_hidden_units_2) + '_' + '.h5'
print(filename)

In [ ]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return (true_positives / (predicted_positives + K.epsilon()))

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return (true_positives / (possible_positives + K.epsilon()))

def f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return ( 2*(precision * recall) / (precision + recall + K.epsilon()) )

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import (Dense, Activation, 
                          Dropout, BatchNormalization, 
                          GlobalAveragePooling2D)
from keras import Model
from keras.models import load_model

inc = InceptionV3(include_top=False, input_shape=(IM_HEIGHT, IM_WIDTH, 3), weights='imagenet')
# x = inc.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)
# x = Dense(num_hidden_units)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.2)(x)
# out = Dense(1, activation='sigmoid')(x)
# inc = load_model('Finalized Hierarchical Models/bcc_others_inceptionv3_512_full_trainbias-pos25.h5', 
#                  custom_objects={'f1': f1, 'precision': precision, 'recall': recall})
# for _ in range(6):
#     inc.layers.pop()
print(inc.layers[-1])
x = inc.layers[-1].output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(num_hidden_units_1, name='d1')(x)
x = BatchNormalization(name='bn1')(x)
x = Activation('relu', name='a1')(x)
# x = Dropout(0.5, name='do1')(x)

# x = Dense(num_hidden_units_1, name='d2')(x)
# x = BatchNormalization(name='bn2')(x)
# x = Activation('relu', name='a2')(x)
x = Dropout(0.2, name='do2')(x)

out = Dense(1, activation='sigmoid', name='out')(x)

model = Model(inc.input, out)

last_weights = model.layers[-1].get_weights()
last_weights[1] += 0.666
model.layers[-1].set_weights(last_weights)

for layer in inc.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
def make_plots(h):
    import matplotlib.pyplot as plt
    %matplotlib inline

    fig = plt.figure()
    plt.plot(h.history['loss'], 'r-')
    plt.plot(h.history['val_loss'], 'b-')
    plt.title('Loss plot')
    plt.legend(['Training loss', 'Validation loss'])
    plt.show()

    fig = plt.figure()
    plt.plot(h.history['f1'], 'r-')
    plt.plot(h.history['val_f1'], 'b-')
    plt.title('F1 plot')
    plt.legend(['Training F1', 'Validation F1'])
    plt.show()

    fig = plt.figure()
    plt.plot(h.history['precision'], 'r-')
    plt.plot(h.history['val_precision'], 'b-')
    plt.title('Precision plot')
    plt.legend(['Training precision', 'Validation precision'])
    plt.show()

    fig = plt.figure()
    plt.plot(h.history['recall'], 'r-')
    plt.plot(h.history['val_recall'], 'b-')
    plt.title('Recall plot')
    plt.legend(['Training recall', 'Validation recall'])
    plt.show()

In [ ]:
from keras.optimizers import Adam
model.compile(loss='binary_crossentropy', optimizer=Adam(6e-5))
_ = model.fit_generator(train_gen, 
              steps_per_epoch=(train_gen.samples//BATCH_SIZE)+1, 
              validation_data=val_gen, 
              validation_steps=(val_gen.samples//BATCH_SIZE)+1, 
              shuffle=True, class_weight=class_weights, epochs=1)

In [ ]:
from keras.optimizers import Adam

for layer in model.layers:
    layer.trainable = True

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=5e-5, decay=8e-4), 
              metrics=['accuracy', precision, recall, f1])
print('Model compiled')

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

stopper = EarlyStopping(monitor='val_loss', mode='min', patience=5, verbose=1)
checkpoint = ModelCheckpoint(filename, 
                             monitor='val_f1', 
                             mode='max', save_best_only=True, 
                             verbose=1)
r = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.05)

hist = model.fit_generator(train_gen, 
                           steps_per_epoch=(train_gen.samples//BATCH_SIZE)+1, 
                           validation_data=val_gen, 
                           validation_steps=(val_gen.samples//BATCH_SIZE)+1, 
                           shuffle=True, class_weight=class_weights, 
                           callbacks=[stopper, checkpoint, r],
                           epochs=20, initial_epoch=initial_epoch)

make_plots(hist)

In [ ]:
from keras.models import load_model

y_true = test_gen.classes
y_pred = model.predict_generator(test_gen,
                                 verbose=1)

print(y_true.shape)
print(y_pred.shape)

y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = 0

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

'''
             precision    recall  f1-score   support

          0       0.99      0.91      0.95       222
          1       0.62      0.94      0.75        33

avg / total       0.94      0.92      0.92       255


----------------

[[203  19]
 [  2  31]]
'''


print(classification_report(y_true=y_true.ravel(), y_pred=y_pred.ravel()))
print('\n----------------\n')
print(confusion_matrix(y_true=y_true.ravel(), y_pred=y_pred.ravel()))

In [ ]:
'''
             precision    recall  f1-score   support

          0       0.98      0.93      0.95       222
          1       0.64      0.88      0.74        33

avg / total       0.94      0.92      0.93       255


----------------

[[206  16]
 [  4  29]]

'''